<b><font color="SteelBlue" size="+3">Anticipez les besoins en consommation de bâtiments 2</font></b>

Ce notebook est la suite du notebook d'exploration des données Deveau_Estelle_1_notebook_exploratoire_022024

# Introduction

## Imports

In [1]:
# Chargement des librairies
# Builtin
import os

# Data
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
import time
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate

# hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

## Data

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'Council District Map - 2024 - FULL.png',
 'data',
 'Deveau_Estelle_1_notebook_exploratoire_022024.ipynb',
 'Deveau_Estelle_2_notebook_prediction_022024.ipynb',
 'Map_of_Seattle,_divided_by_districts.png',
 'plan-codes-postaux -seattle.jpg']

In [3]:
os.listdir("data/cleaned/")

['df_cleaned.csv']

In [4]:
path     = "./data/cleaned/"
filename = "df_cleaned.csv"

In [5]:
df = pd.read_csv(path + filename)
df.head()

,Log_TotalGHGEmissions,Log_SiteEnergyUseWN,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA,BuildingAge,GroupedPrimType,GroupedNeighborhood,GroupedLargType,SteamUse_pct,Electricity_pct,NaturalGas_pct,ENERGYSTARScore
0,5.521381,15.824652,1.0,12,88434,88434.0,89,B,C,C,26.872820,52.917723,17.117720,60.0
1,5.689886,15.974742,1.0,11,103566,83880.0,20,B,C,C,0.000000,37.426959,59.381320,61.0
2,5.657494,15.753792,1.0,10,61320,61320.0,90,B,C,C,31.877211,39.858983,26.072621,56.0
3,5.171279,15.617677,1.0,11,83008,81352.0,90,B,C,C,0.000000,46.368511,48.617731,27.0
4,5.400468,15.771071,1.0,8,102761,102761.0,90,B,C,C,32.206065,51.453254,5.449851,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987 entries, 0 to 986
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Log_TotalGHGEmissions      987 non-null    float64
 1   Log_SiteEnergyUseWN        987 non-null    float64
 2   NumberofBuildings          987 non-null    float64
 3   NumberofFloors             987 non-null    int64  
 4   PropertyGFATotal           987 non-null    int64  
 5   LargestPropertyUseTypeGFA  987 non-null    float64
 6   BuildingAge                987 non-null    int64  
 7   GroupedPrimType            987 non-null    object 
 8   GroupedNeighborhood        987 non-null    object 
 9   GroupedLargType            987 non-null    object 
 10  SteamUse_pct               987 non-null    float64
 11  Electricity_pct            987 non-null    float64
 12  NaturalGas_pct             987 non-null    float64
 13  ENERGYSTARScore            620 non-null    float64

In [7]:
df.describe()

,Log_TotalGHGEmissions,Log_SiteEnergyUseWN,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA,BuildingAge,SteamUse_pct,Electricity_pct,NaturalGas_pct,ENERGYSTARScore
count,987.000000,987.000000,987.000000,987.000000,9.870000e+02,9.870000e+02,987.000000,987.000000,987.000000,987.000000,620.000000
mean,3.781659,14.736482,1.166160,3.322188,8.006021e+04,7.122909e+04,59.987842,1.899422,64.221547,28.311586,62.488710
std,1.478016,1.224005,1.393534,5.381731,1.395180e+05,1.241587e+05,31.015763,8.432537,27.327938,24.227952,28.852612
min,-0.916291,10.970165,1.000000,1.000000,1.128500e+04,6.601000e+03,1.000000,0.000000,0.000000,0.000000,1.000000
25%,2.883123,13.919732,1.000000,1.000000,2.626650e+04,2.343350e+04,37.000000,0.000000,41.785456,0.000000,43.000000
50%,3.765377,14.588011,1.000000,2.000000,3.997100e+04,3.569600e+04,56.000000,0.000000,62.004494,28.271670,69.000000
75%,4.768464,15.501170,1.000000,3.000000,7.331550e+04,6.476600e+04,88.000000,0.000000,93.488903,48.695422,87.000000
max,9.005223,19.431285,27.000000,76.000000,1.952220e+06,1.680937e+06,116.000000,63.453768,103.872233,97.077764,100.000000


Nous allons commencer par chercher un modèle de ML pour prédire le SiteEnergyUseWN

Dans un premier temps, nous allons travailler en excuant l'ENERGYSTARScore puis nous ferons une comparaison avec son utilisation.

# Modelisation sans l'Energy Star Score

## Data preparation

### Séparation des données

In [8]:
# Sélection des variables explicatives et des variables cibles
features = df.drop(['Log_TotalGHGEmissions', 'Log_SiteEnergyUseWN', 'ENERGYSTARScore'], axis=1)
target = df['Log_SiteEnergyUseWN']

# Transformation des variables catégorielles en variables numériques (encodage one-hot)
features = pd.get_dummies(features)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
# Conservez les noms des colonnes dans une variable avant la mise à l'échelle
column_names = X_train.columns

# Affichage des dimensions des ensembles d'entraînement et de test
X_train.shape, X_test.shape

((789, 19), (198, 19))

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 789 entries, 756 to 102
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   NumberofBuildings          789 non-null    float64
 1   NumberofFloors             789 non-null    int64  
 2   PropertyGFATotal           789 non-null    int64  
 3   LargestPropertyUseTypeGFA  789 non-null    float64
 4   BuildingAge                789 non-null    int64  
 5   SteamUse_pct               789 non-null    float64
 6   Electricity_pct            789 non-null    float64
 7   NaturalGas_pct             789 non-null    float64
 8   GroupedPrimType_A          789 non-null    bool   
 9   GroupedPrimType_B          789 non-null    bool   
 10  GroupedPrimType_C          789 non-null    bool   
 11  GroupedPrimType_D          789 non-null    bool   
 12  GroupedNeighborhood_A      789 non-null    bool   
 13  GroupedNeighborhood_B      789 non-null    bool   
 1

In [10]:
X_train.head()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA,BuildingAge,SteamUse_pct,Electricity_pct,NaturalGas_pct,GroupedPrimType_A,GroupedPrimType_B,GroupedPrimType_C,GroupedPrimType_D,GroupedNeighborhood_A,GroupedNeighborhood_B,GroupedNeighborhood_C,GroupedLargType_A,GroupedLargType_B,GroupedLargType_C,GroupedLargType_D
756,1.0,1,37247,37247.0,52,0.0,37.624994,50.333423,False,False,True,False,False,True,False,False,False,False,True
716,1.0,1,24295,24295.0,70,0.0,95.173696,0.000000,False,False,False,True,True,False,False,False,False,False,True
49,1.0,11,299070,250000.0,12,0.0,46.441677,52.553296,False,True,False,False,False,False,True,False,False,True,False
718,1.0,1,24377,24377.0,43,0.0,56.222920,40.987709,True,False,False,False,True,False,False,True,False,False,False
371,1.0,3,29636,25096.0,95,0.0,100.000027,0.000000,False,True,False,False,True,False,False,False,False,True,False


In [11]:
y_train.head()

756    14.502657
716    12.333033
49     18.005170
718    16.131400
371    13.442477
Name: Log_SiteEnergyUseWN, dtype: float64

### Standardisation

In [12]:
# Initialisation du StandardScaler
scaler = StandardScaler()

In [13]:
columns_to_scale=['NumberofBuildings','NumberofFloors',	'PropertyGFATotal', 'LargestPropertyUseTypeGFA', 
                  'BuildingAge', 'SteamUse_pct', 'Electricity_pct', 'NaturalGas_pct']

In [14]:
# Séparer les colonnes à normaliser
X_train_to_scale = X_train[columns_to_scale]
X_test_to_scale = X_test[columns_to_scale]

# Appliquer la normalisation sur ces colonnes
X_train_scaled = scaler.fit_transform(X_train_to_scale)
X_test_scaled = scaler.transform(X_test_to_scale)

# Merge
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=columns_to_scale, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=columns_to_scale, index=X_test.index)
X_train_final = X_train.drop(columns=columns_to_scale).join(X_train_scaled_df)
X_test_final = X_test.drop(columns=columns_to_scale).join(X_test_scaled_df)


In [15]:
X_train_final.head()

,GroupedPrimType_A,GroupedPrimType_B,GroupedPrimType_C,GroupedPrimType_D,GroupedNeighborhood_A,GroupedNeighborhood_B,GroupedNeighborhood_C,GroupedLargType_A,GroupedLargType_B,GroupedLargType_C,GroupedLargType_D,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA,BuildingAge,SteamUse_pct,Electricity_pct,NaturalGas_pct
756,False,False,True,False,False,True,False,False,False,False,True,-0.116665,-0.458673,-0.332165,-0.295953,-0.262960,-0.224625,-0.971908,0.908342
716,False,False,False,True,True,False,False,False,False,False,True,-0.116665,-0.458673,-0.431800,-0.409088,0.309825,-0.224625,1.145041,-1.173894
49,False,True,False,False,False,False,True,False,False,True,False,-0.116665,1.480731,1.681941,1.562423,-1.535814,-0.224625,-0.647583,1.000176
718,True,False,False,False,True,False,False,True,False,False,False,-0.116665,-0.458673,-0.431169,-0.408372,-0.549352,-0.224625,-0.287777,0.521721
371,False,True,False,False,True,False,False,False,False,True,False,-0.116665,-0.070792,-0.390714,-0.402091,1.105359,-0.224625,1.322579,-1.173894


## Préparation des métriques

In [16]:
# Fonction pour calculer les métriques
def calc_metrics(y_true, y_pred):
    # Calcul du RMSE 
    rmse_log = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # Calcul du R-squared 
    r2_log = r2_score(y_true, y_pred)
    
    # Calcul du MAE 
    mae_log = mean_absolute_error(y_true, y_pred)
    
    return rmse_log, r2_log, mae_log

# Fonction pour afficher les métriques
def display_metrics(rmse_log, r2_log, mae_log):
    # Affichage des métriques
    print(f"RMSE : {rmse_log}")
    print(f"R-squared : {r2_log}")
    print(f"MAE : {mae_log}")

## Tests de modèles

### Régression linéaire (Baseline)

In [17]:
# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred = model.predict(X_test_final)

print("Métriques pour le modèle moyen :")
metrics = calc_metrics(y_test, y_pred)
display_metrics(*metrics) 


Métriques pour le modèle moyen :
RMSE : 0.8783934435509133
R-squared : 0.45153864992624615
MAE : 0.6345306593759301


### SVR

In [18]:
# Démarre le chronomètre
start_time = time.time()

# Création du modèle SVM pour la régression
svr_model = SVR()

# Entraînement du modèle sur les données d'entraînement
svr_model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred_svr = svr_model.predict(X_test_final)

# Arrête le chronomètre
end_time = time.time()

# Calcule la durée totale
duration = end_time - start_time

print(f"Le temps de calcul est de {duration:.2f} secondes")
print("Métriques pour Support Vector Regression:")
metrics = calc_metrics(y_test, y_pred_svr)
display_metrics(*metrics)

Le temps de calcul est de 0.03 secondes
Métriques pour Support Vector Regression:
RMSE : 0.6824707081683659
R-squared : 0.6689177564087858
MAE : 0.4952514128582784


### Forêt aléatoire

In [19]:
# Démarre le chronomètre
start_time = time.time()

# Création du modèle Random Forest
rf_model = RandomForestRegressor()

# Entraînement du modèle sur les données d'entraînement
rf_model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred_rf = rf_model.predict(X_test_final)

# Arrête le chronomètre
end_time = time.time()

# Calcule la durée totale
duration = end_time - start_time

print(f"Le temps de calcul est de {duration:.2f} secondes")
print("Métriques pour Random Forest:")
metrics = calc_metrics(y_test, y_pred_rf)
display_metrics(*metrics) 

Le temps de calcul est de 0.57 secondes
Métriques pour Random Forest:
RMSE : 0.686112878310881
R-squared : 0.6653745258377206
MAE : 0.48982430106541647


In [20]:
importances_rf = rf_model.feature_importances_

# Création d'un DataFrame pour afficher l'importance des variables
features_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances_rf})
features_rf = features_rf.sort_values(by='Importance', ascending=False)

print(features_rf)

                      Feature  Importance
13      GroupedNeighborhood_B    0.568694
3   LargestPropertyUseTypeGFA    0.078341
14      GroupedNeighborhood_C    0.063510
18          GroupedLargType_D    0.057299
17          GroupedLargType_C    0.052406
15          GroupedLargType_A    0.047339
7              NaturalGas_pct    0.041884
0           NumberofBuildings    0.020437
12      GroupedNeighborhood_A    0.015756
10          GroupedPrimType_C    0.015273
8           GroupedPrimType_A    0.006404
2            PropertyGFATotal    0.006232
5                SteamUse_pct    0.005496
9           GroupedPrimType_B    0.004974
6             Electricity_pct    0.004450
4                 BuildingAge    0.003910
1              NumberofFloors    0.003431
11          GroupedPrimType_D    0.002446
16          GroupedLargType_B    0.001719


### Gradient Boosting

In [21]:
# Démarre le chronomètre
start_time = time.time()

# Création du modèle Gradient Boosting
gb_model = GradientBoostingRegressor()

# Entraînement du modèle sur les données d'entraînement
gb_model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred_gb = gb_model.predict(X_test_final)

# Arrête le chronomètre
end_time = time.time()

# Calcule la durée totale
duration = end_time - start_time

print(f"Le temps de calcul est de {duration:.2f} secondes")
print("Métriques pour Gradient Boosting:")
metrics = calc_metrics(y_test, y_pred_gb)
display_metrics(*metrics) 

Le temps de calcul est de 0.20 secondes
Métriques pour Gradient Boosting:
RMSE : 0.6198374284381443
R-squared : 0.7268988984191802
MAE : 0.44664875187394165


In [22]:
importances_gb = gb_model.feature_importances_

# Création d'un DataFrame pour afficher l'importance des variables
features_gb = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances_gb})
features_gb = features_gb.sort_values(by='Importance', ascending=False)

print(features_gb)

                      Feature  Importance
13      GroupedNeighborhood_B    0.623529
3   LargestPropertyUseTypeGFA    0.105682
18          GroupedLargType_D    0.052590
7              NaturalGas_pct    0.051577
17          GroupedLargType_C    0.043651
10          GroupedPrimType_C    0.032668
14      GroupedNeighborhood_C    0.025893
15          GroupedLargType_A    0.022823
0           NumberofBuildings    0.016333
5                SteamUse_pct    0.004412
8           GroupedPrimType_A    0.004060
12      GroupedNeighborhood_A    0.003587
1              NumberofFloors    0.003037
11          GroupedPrimType_D    0.002967
9           GroupedPrimType_B    0.002757
16          GroupedLargType_B    0.001829
2            PropertyGFATotal    0.001278
6             Electricity_pct    0.001133
4                 BuildingAge    0.000197


<b>Modèle moyen</b> : <br>
Le modèle moyen sert de point de référence minimal pour évaluer les performances des autres modèles.

<b>Forêt Aléatoire</b> :<br>
Le modèle de forêt aléatoire a surpassé de manière significative la régression linéaire, avec un RMSE plus faible et un R-squared positif.
    
<b>Gradient Boosting</b> :<br>
Le modèle Gradient Boosting offre une amélioration légère mais significative sur la forêt aléatoire en termes de RMSE et de R-squared. 
Le Gradient Boosting est également plus rapide à calculer.

<b>Résumé</b> :
   - Le modèle moyen sert d'étalon de base, mais ses performances sont clairement surpassées par des modèles plus complexes.
   - La forêt aléatoire et le Gradient Boosting émergent comme les meilleurs candidats pour résoudre ce problème, avec un  avantage pour le gradient boosting.
 
    
Compte tenu de ces résultats, nous allons procéder à une validation croisée des modèles de Gradient Boosting et Random Forest pour confirmer le meilleur modèle.

## Validation croisée

In [23]:
# Métriques à utiliser
scoring = ['neg_mean_squared_error', 'r2', 'neg_mean_absolute_error']

# Pour Random Forest
rf_scores = cross_validate(rf_model, X_train_final, y_train, cv=5, scoring=scoring)
rf_rmse = np.sqrt(-rf_scores['test_neg_mean_squared_error'].mean())
rf_r2 = rf_scores['test_r2'].mean()
rf_mae = -rf_scores['test_neg_mean_absolute_error'].mean()

# Pour Gradient Boosting
gb_scores = cross_validate(gb_model, X_train_final, y_train, cv=5, scoring=scoring)
gb_rmse = np.sqrt(-gb_scores['test_neg_mean_squared_error'].mean())
gb_r2 = gb_scores['test_r2'].mean()
gb_mae = -gb_scores['test_neg_mean_absolute_error'].mean()

# Création d'un DataFrame pour afficher les métriques de Random Forest et Gradient Boosting
df_metrics = pd.DataFrame({
    'Metric': ['RMSE', 'R2', 'MAE'],
    'Random Forest': [rf_rmse, rf_r2, rf_mae],
    'Gradient Boosting': [gb_rmse, gb_r2, gb_mae]
})

# Afficher le DataFrame
df_metrics


,Metric,Random Forest,Gradient Boosting
0,RMSE,0.657835,0.630360
1,R2,0.705204,0.728783
2,MAE,0.497445,0.482901


La différence entre les métriques des deux modèles sont suffisamment significative pour confirmer le choix du Gradient Boosting.

## Recherche des hyperparamètres

In [24]:
# Définition des hyperparamètres à tester
param_dist = {
    'n_estimators': np.arange(50, 200, 10),
    'learning_rate': np.linspace(0.008, 0.2, 20),
    'max_depth': np.arange(2, 10, 1),
    'min_samples_split': np.arange(2, 10, 1),
    'min_samples_leaf': np.arange(1, 5, 1),
    'subsample': np.linspace(0.5, 0.9, 20)
}

# Initialisation de la recherche aléatoire
random_search = RandomizedSearchCV(GradientBoostingRegressor(), param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Exécution de la recherche aléatoire sur le jeu d'entraînement
random_search.fit(X_train_final, y_train)

# Récupération du meilleur modèle
best_gb_model = random_search.best_estimator_

# Affichage des meilleurs hyperparamètres
print(random_search.best_params_)

{'subsample': 0.6052631578947368, 'n_estimators': 130, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_depth': 3, 'learning_rate': 0.1191578947368421}


In [25]:
# Définition de la grille d'hyperparamètres
param_grid = {
    'n_estimators': [200, 250, 260],
    'learning_rate': [0.04, 0.05],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3, 4],
    'subsample': [0.5, 0.6, 0.7]
}

# Configuration de GridSearchCV
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Exécution de la recherche d'hyperparamètres
grid_search.fit(X_train_final, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres :", grid_search.best_params_)

# Prédiction avec le meilleur modèle
y_pred = grid_search.best_estimator_.predict(X_test_final)

# Calcul des métriques pour le meilleur modèle
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("RMSE :", rmse)
print("R² :", r2)
print("MAE :", mae)

# Affichage des 3 meilleures configurations d'hyperparamètres
results = grid_search.cv_results_
for i in range(3):
    print(f"\nConfiguration {i+1}:")
    print("Paramètres :", results['params'][results['rank_test_score'][i]])
    print("RMSE :", np.sqrt(-results['mean_test_score'][results['rank_test_score'][i]]))

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Meilleurs paramètres : {'learning_rate': 0.05, 'max_depth': 3, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 250, 'subsample': 0.5}
RMSE : 0.614217865449577
R² : 0.7318284228158335
MAE : 0.4436127026944857

Configuration 1:
Paramètres : {'learning_rate': 0.04, 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 250, 'subsample': 0.7}
RMSE : 0.6379375815991001
Std Dev : 0.0596178167175468

Configuration 2:
Paramètres : {'learning_rate': 0.05, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 200, 'subsample': 0.5}
RMSE : 0.6206064364633386
Std Dev : 0.05418712182945085

Configuration 3:
Paramètres : {'learning_rate': 0.05, 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 260, 'subsample': 0.5}
RMSE : 0.6304925289322254
Std Dev : 0.05792368572381668
